In [58]:
'''
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

'''

'\n!apt-get install -y -qq software-properties-common python-software-properties module-init-tools\n!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null\n!apt-get update -qq 2>&1 > /dev/null\n!apt-get -y install -qq google-drive-ocamlfuse fuse\nfrom google.colab import auth\nauth.authenticate_user()\nfrom oauth2client.client import GoogleCredentials\ncreds = GoogleCredentials.get_application_default()\nimport getpass\n!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL\nvcode = getpass.getpass()\n!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}\n\n'

In [59]:
'''
!mkdir -p drive
!google-drive-ocamlfuse drive
'''

'\n!mkdir -p drive\n!google-drive-ocamlfuse drive\n'

In [60]:
'''
from google.colab import drive
drive.mount('/content/drive')
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [61]:
'''
!pip install torchvision
'''

'\n!pip install torchvision\n'

In [62]:
!nvidia-smi

Fri Aug 13 12:51:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    34W / 250W |   2805MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [63]:
#!pip install mxnet-cu112 # 没用着呢
#!pip install minpy

In [64]:
import torch 
x = torch.empty(5,3)
print(x)

tensor([[-3.4530e+30,  3.0820e-41,  0.0000e+00],
        [ 0.0000e+00,  1.4013e-45,  0.0000e+00],
        [-1.6330e+28,  3.0820e-41,  0.0000e+00],
        [ 0.0000e+00, -2.6008e-18,  4.5705e-41],
        [ 0.0000e+00,  0.0000e+00, -3.2537e-35]])


# main.py 前面import

In [65]:
# -*- coding: UTF-8 -*-
# Python
import os
import random

# Torch
import torch
import numpy as np  #注意这里可以改用Gpu的Numpy
import torch.nn as nn
import torch.optim as optim
from numpy.core._multiarray_umath import ndarray
from sklearn.cluster import KMeans
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.autograd import Variable
from torch.distributions import Categorical

# Torchvison
import torchvision.transforms as T
import torchvision.models as models
from torchvision.datasets import CIFAR100, CIFAR10
# from influence import *

# Utils
# import visdom
# from tqdm import tqdm

# Custom
#import models.resnet as resnet
#from config import *
#from data.sampler import SubsetSequentialSampler

# import copy

#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"


# config.py

In [66]:
##
# Learning Loss for Active Learning

# Config
NUM_TRAIN = 50000 # N 训练集数据个数
NUM_VAL   = 50000 - NUM_TRAIN
BATCH     = 128  # 128 # 每个dataloader的图片个数
SUBSET    = 25000 # M  # 每次在未标签的数据集中选来聚类的图片个数
ADDENDUM  = 2500  # K  # 每个cycle新加入的不确定度最大的图片个数

MARGIN = 1.0  # xi
WEIGHT = 1.0  # 1.0 # lambda

TRIALS = 1 #尝试 一次
CYCLES = 7  #迭代数

EPOCH = 200   #神经网络训练轮数
LR = 0.1    # 0.1 for SGD # 初始学习速率
MILESTONES = [160] 
EPOCHL = 120 # 

MOMENTUM = 0.9 # optimizer优化器参数之一
WDECAY = 5e-4 # optimizer优化器参数之一

CLUSTER_NUMS = 10 # Kmeans 选取的K值
CLUSTER_MAX_ITER = 10000 #聚类最大迭代次数


''' CIFAR-10 | ResNet-18 | 93.6%
NUM_TRAIN = 50000 # N
NUM_VAL   = 50000 - NUM_TRAIN
BATCH     = 128 # B
SUBSET    = NUM_TRAIN # M
ADDENDUM  = NUM_TRAIN # K

MARGIN = 1.0 # xi
WEIGHT = 0.0 # lambda

TRIALS = 1
CYCLES = 1

EPOCH = 50
LR = 0.1
MILESTONES = [25, 35]
EPOCHL = 40

MOMENTUM = 0.9
WDECAY = 5e-4
'''


' CIFAR-10 | ResNet-18 | 93.6%\nNUM_TRAIN = 50000 # N\nNUM_VAL   = 50000 - NUM_TRAIN\nBATCH     = 128 # B\nSUBSET    = NUM_TRAIN # M\nADDENDUM  = NUM_TRAIN # K\n\nMARGIN = 1.0 # xi\nWEIGHT = 0.0 # lambda\n\nTRIALS = 1\nCYCLES = 1\n\nEPOCH = 50\nLR = 0.1\nMILESTONES = [25, 35]\nEPOCHL = 40\n\nMOMENTUM = 0.9\nWDECAY = 5e-4\n'

# device设置成cuda或GPU 

In [67]:
#
# 加一个device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


# sampler.py

In [68]:
class SubsetSequentialSampler(torch.utils.data.Sampler):
    r"""Samples elements sequentially from a given list of indices, without replacement.

    Arguments:
        indices (sequence): a sequence of indices
    """

    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))

    def __len__(self):
        return len(self.indices)



# main.py 里面的data

In [69]:
##
# Data
train_transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomCrop(size=32, padding=4),
    T.ToTensor(),
    T.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

test_transform = T.Compose([
    T.ToTensor(),
    T.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

cifar10_train = CIFAR10('../cifar10', train=True, download=True, transform=train_transform)  # specify data path here
cifar10_unlabeled = CIFAR10('../cifar10', train=True, download=True, transform=test_transform)
cifar10_test = CIFAR10('../cifar10', train=False, download=True, transform=test_transform)

##
# Train Utils
iters = 0


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# resnet.py

In [70]:
#import torch
#import torch.nn as nn
#import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
        self.linear1 = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out1 = self.layer1(out)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        out4 = self.layer4(out3)
        out5 = F.avg_pool2d(out4, 4)
        out5 = out5.view(out5.size(0), -1)     # [128, 512]
        out = self.linear(out5)
        out_cons = self.linear1(out5)

        return out, out_cons, out5, [out1, out2, out3, out4]


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet18_student():
    return ResNet(BasicBlock, [1,1,1,1])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])



# main.py 里面的train_epoch函数

In [71]:
#
def train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss):
    models['backbone'].train()
    global iters

    for data in dataloaders['train']:
        #inputs = data[0].cuda()这是pycharm原来的
        inputs = data[0].to(device)
        #labels = data[1].cuda()这是pycharm原来的
        labels = data[1].to(device)
        iters += 1

        optimizers['backbone'].zero_grad()

        scores, _, _, features_list = models['backbone'](inputs)
        target_loss = criterion(scores, labels)

        m_backbone_loss = torch.sum(target_loss) / target_loss.size(0)
        loss = m_backbone_loss
        loss.backward()
        optimizers['backbone'].step()


# main.py 里面的test函数

In [72]:
#
def test(models, dataloaders, mode='val'):
    assert mode == 'val' or mode == 'test'
    models['backbone'].eval()

    total = 0
    correct = 0
    with torch.no_grad():
        for (inputs, labels) in dataloaders[mode]:
            #inputs = inputs.cuda()这才是prcharm原来的
            inputs = inputs.to(device)
            #labels = labels.cuda()这才是prcharm原来的
            labels = labels.to(device)
            scores, _, _, _ = models['backbone'](inputs)
            _, preds = torch.max(scores.data, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    return 100 * correct / total

# main.py里面的train函数

In [73]:
#
def train(models, criterion, optimizers, schedulers, dataloaders, num_epochs, epoch_loss, cycle):
    print('>> Train a Model...')
    best_acc = 0.
    checkpoint_dir = os.path.join('./cifar10', 'train', 'weights')
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    for epoch in range(num_epochs):

        train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss)
        schedulers['backbone'].step()

        # Save a checkpoint
        if epoch % 20 == 0 or epoch == EPOCH - 1:
            acc = test(models, dataloaders, 'test')
            if best_acc < acc:
                best_acc = acc
                torch.save({
                    'epoch': epoch + 1,
                    'state_dict_backbone': models['backbone'].state_dict()
                    # 'state_dict_module': models['module'].state_dict()
                },
                    '%s/active_resnet18_cifar10.pth' % (checkpoint_dir))
            print('Cycle:', cycle + 1, 'Epoch:', epoch + 1, "---", 'Val Acc: {:.3f} \t Best Acc: {:.3f}'.format(acc, best_acc),flush=True)
            
    print('>> Finished.')


# MMD

In [74]:
def guassian_kernel(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    """计算Gram核矩阵
    source: sample_size_1 * feature_size 的数据
    target: sample_size_2 * feature_size 的数据
    kernel_mul: 这个概念不太清楚，感觉也是为了计算每个核的bandwith
    kernel_num: 表示的是多核的数量
    fix_sigma: 表示是否使用固定的标准差
        return: (sample_size_1 + sample_size_2) * (sample_size_1 + sample_size_2)的
                        矩阵，表达形式:
                        [   K_ss K_st
                            K_ts K_tt ]
    """
    n_samples = int(source.size()[0])+int(target.size()[0])
    total = torch.cat([source, target], dim=0) # 合并在一起

    total0 = total.unsqueeze(0).expand(int(total.size(0)), \
                                       int(total.size(0)), \
                                       int(total.size(1)))
    total1 = total.unsqueeze(1).expand(int(total.size(0)), \
                                       int(total.size(0)), \
                                       int(total.size(1)))
    L2_distance = ((total0-total1)**2).sum(2) # 计算高斯核中的|x-y|

    # 计算多核中每个核的bandwidth
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
    bandwidth /= kernel_mul ** (kernel_num // 2)
    bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]

    # 高斯核的公式，exp(-|x-y|/bandwith)
    kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for \
                  bandwidth_temp in bandwidth_list]

    return sum(kernel_val) # 将多个核合并在一起

def MMD(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    n = int(source.size()[0])
    m = int(target.size()[0])

    kernels = guassian_kernel(source, target,
                              kernel_mul=kernel_mul, kernel_num=kernel_num, fix_sigma=fix_sigma)
    XX = kernels[:n, :n] 
    YY = kernels[n:, n:]
    XY = kernels[:n, n:]
    YX = kernels[n:, :n]

    XX = torch.div(XX, n * n).sum(dim=1).view(1,-1)  # K_ss矩阵，Source<->Source
    XY = torch.div(XY, -n * m).sum(dim=1).view(1,-1) # K_st矩阵，Source<->Target

    YX = torch.div(YX, -m * n).sum(dim=1).view(1,-1) # K_ts矩阵,Target<->Source
    YY = torch.div(YY, m * m).sum(dim=1).view(1,-1)  # K_tt矩阵,Target<->Target
    	
    loss = (XX + XY).sum() + (YX + YY).sum()
    return loss
    
'''
if __name__ == "__main__":
    # 样本数量可以不同，特征数目必须相同

    # 100和90是样本数量，50是特征数目
    data_1 = torch.tensor(np.random.normal(loc=0,scale=10,size=(100,50)))
    data_2 = torch.tensor(np.random.normal(loc=10,scale=10,size=(90,50)))
    print("MMD Loss:",mmd(data_1,data_2))

    data_1 = torch.tensor(np.random.normal(loc=0,scale=10,size=(100,50)))
    data_2 = torch.tensor(np.random.normal(loc=0,scale=9,size=(80,50)))

    print("MMD Loss:",mmd(data_1,data_2))

'''

'\nif __name__ == "__main__":\n    # 样本数量可以不同，特征数目必须相同\n\n    # 100和90是样本数量，50是特征数目\n    data_1 = torch.tensor(np.random.normal(loc=0,scale=10,size=(100,50)))\n    data_2 = torch.tensor(np.random.normal(loc=10,scale=10,size=(90,50)))\n    print("MMD Loss:",mmd(data_1,data_2))\n\n    data_1 = torch.tensor(np.random.normal(loc=0,scale=10,size=(100,50)))\n    data_2 = torch.tensor(np.random.normal(loc=0,scale=9,size=(80,50)))\n\n    print("MMD Loss:",mmd(data_1,data_2))\n\n'

# main.py里面的get_uncertainty函数

In [75]:
#
def get_uncertainty(models, labeled_loader, unlabeled_loader):
    models['backbone'].eval()
    labeled_features = torch.tensor([]).to(device)
    unlabeled_features = torch.tensor([]).to(device)
    target_features = torch.tensor([]).to(device) #每个 batch 从 unlabeled_features里随机选择的200个feature
    samples_chosen = torch.tensor([]) #最终选出的样本，所在位置会标为1，其余为0

    with torch.no_grad():
        print(">> Start MMD:")
        
        # 提取labeled_set里面已训练过数据的labeled_features, unlabeled_set里数据的feature来求uncertainty
        for (inputs, labels) in labeled_loader:
            inputs = inputs.to(device)
            _, _, feature_1, _= models['backbone'](inputs)
            # feature_1: [batch_size,512], 行:batch_size，即图片个数，512是feature个数
            labeled_features = torch.cat((labeled_features, feature_1), axis = 0) 
            #labeled_features.cpu()  # labeled_features:　Ｔ

        for (inputs, labels) in unlabeled_loader:
            inputs = inputs.to(device)
            _, _, feature_2, _ = models['backbone'](inputs)
            # feature_2: [batch_size,512], 行:batch_size，即图片个数，512是feature个数
            unlabeled_features = torch.cat((unlabeled_features, feature_2), axis = 0)
            #unlabeled_features.cpu()
        
        
        batch_num = 0
        for (inputs, labels) in unlabeled_loader:
            inputs = inputs.to(device)
            _, _, batch_features, _ = models['backbone'](inputs)
            
            source_features = torch.tensor([]).to(device)
            
            #每个 batch 的 MMD分数和 uncertainty 分数 
            MMD_batch_score = torch.tensor([])
            uncertainty_score = torch.tensor([])
            
            batch_num = batch_num + 1
            
            #算 (T+ batch_features里的每一个single_feature) 与 target_features 的 MMD
            for single_feature in batch_features:
                #print(feature.shape)
                #labeled_features.to(device)
                #target_features.to(device)
                five_MMD_loss = []
                
                for i in range(5):
                    index_sampled_1 = random.sample(range(len(unlabeled_features)),200)
                    #random.shuffle(unlabeled_features)
                    target_features = unlabeled_features[index_sampled_1] #每个 batch 从 unlabeled_features里随机选择的200个feature
                    #print(target_features)
                    #random.shuffle(labeled_features)
                    index_sampled_2 = random.sample(range(len(labeled_features)),200)
                    T_features = labeled_features[index_sampled_2]
                    source_features = torch.cat((T_features, single_feature.reshape(1,512).to(device)), axis = 0) 
                    MMD_gpu = MMD(source_features, target_features)
                    MMD_loss = MMD_gpu.cpu()
                    five_MMD_loss.append(MMD_loss)

                MMD_batch_torch = torch.mean(torch.from_numpy(np.array(five_MMD_loss)))
                MMD_batch_score = torch.cat((MMD_batch_score, MMD_batch_torch.reshape(1,1)), axis = 0)

        
            #把每个batch 的features 用聚类+距离差求uncertainty

            input_x = np.array(batch_features.cpu())   
            # input_x: shape: [BATCH_SIZE,512]
            
            line_num, column_num = input_x.shape  # line_num：样本数量，column_num：每个样本的属性值个数
            
            result = np.empty(line_num, dtype=np.int)  # line_num个样本的聚类结果
            distance = np.empty((line_num, CLUSTER_NUMS), dtype=np.float32)  
            
            # 从line_num个数据样本中不重复地随机选择k个样本作为质心
            cores = input_x[np.random.choice(np.arange(line_num), CLUSTER_NUMS, replace=False)]
            min_distance_differ = np.empty((line_num, 1), dtype=np.float32)  
            iter = 0
            
            while True:  # 迭代聚类计算,max=10000次，质心不变时break
                iter = iter + 1
                d = np.square(np.repeat(input_x, CLUSTER_NUMS, axis=0).reshape(line_num, CLUSTER_NUMS, column_num) 
                            - cores)
                distance = np.sqrt(np.sum(d, axis=2))  # ndarray(line_num, k)，每个样本距离k个质心的距离，共有line_num行

                index_min = np.argmin(distance, axis=1)  # 每个样本距离最近的质心索引序号 [line_num, 1]
                if (index_min == result).all() or iter == CLUSTER_MAX_ITER:
                    break;
                    
                result[:] = index_min  # 重新分类 [line_num, 1] 这些图片分别属于哪个cluster, index
                for i in range(CLUSTER_NUMS):  # 遍历质心集
                    if len(input_x[result == i]) != 0:
                        items = input_x[result == i]  # 找出对应当前质心的子样本集
                        cores[i] = np.mean(items, axis=0)  # 以子样本集的均值作为当前质心的位置

            sorted_distance = np.sort(distance) # 将矩阵的每一行升序排列
            
            min_distance_differ = np.abs(sorted_distance[:, 0].reshape(line_num, 1)
                                            - sorted_distance[:, 1].reshape(line_num, 1))
            
            min_distance_differ_torch = torch.from_numpy(min_distance_differ)
            
            uncertainty_score = 1.00 / min_distance_differ_torch
            MMD_inverse_score = 0.100 / MMD_batch_score 

            total_batch_score = uncertainty_score + MMD_inverse_score   #每个batch各个样本的总分数
            
            #print("batch_num: ", batch_num)
            #print("uncertainty_score: ",uncertainty_score.T)
            #print("MMD_batch_score: ", MMD_inverse_score.T)
            #print("total_batch_score: ", total_batch_score.T)

            # 开始排序+选择
            index_score = torch.zeros(line_num)
            arg = np.argsort(total_batch_score.T).numpy().tolist()
            
            if batch_num == 196:
                chosen_index = arg[0][-4:]
            else:
                chosen_index = arg[0][-13:]
            
            #print("arg: ", arg)
            #print("chosen_index: ",chosen_index.T)
            index_score[chosen_index] = 1
            #print("index_score: ", index_score.T)
            
            samples_chosen = torch.cat((samples_chosen, index_score.reshape(line_num, 1)), axis = 0)
                
    print(">> Clustering Over:")
    return samples_chosen.cpu()


# main.py 里面的main函数

In [ ]:

##
# Main
if __name__ == '__main__':

    for trial in range(TRIALS):
        # Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
        indices = list(range(NUM_TRAIN))
        random.shuffle(indices)
        START = 2 * ADDENDUM
        labeled_set = indices[:START]
        unlabeled_set = indices[START:]

        train_loader = DataLoader(cifar10_train, batch_size=BATCH,  # BATCH
                                  sampler=SubsetRandomSampler(labeled_set),
                                  pin_memory=True)
        test_loader = DataLoader(cifar10_test, batch_size=BATCH)

        dataloaders = {'train': train_loader, 'test': test_loader}

        # Model
        #backbone_net = resnet.ResNet18().cuda() #记住这才是原本pycharm里面的
        backbone_net = ResNet18().to(device)
        
        models = {'backbone': backbone_net}
        torch.backends.cudnn.benchmark = True

        # Active learning cycles
        for cycle in range(CYCLES):
            # Loss, criterion and scheduler (re)initialization
            criterion = nn.CrossEntropyLoss(reduction='none')

            optim_backbone = optim.SGD(models['backbone'].parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WDECAY)

            sched_backbone = lr_scheduler.MultiStepLR(optim_backbone, milestones=MILESTONES)

            optimizers = {'backbone': optim_backbone}
            schedulers = {'backbone': sched_backbone}

            # Training and test
            train(models, criterion, optimizers, schedulers, dataloaders, EPOCH, EPOCHL, cycle)
            acc = test(models, dataloaders, mode='test')
            print('Trial {}/{} || Cycle {}/{} || Label set size {}: Test acc {}'.format(trial + 1, TRIALS, cycle + 1,
                                                                                        CYCLES, len(labeled_set), acc),
                                                                                         flush=True)

            ##
            #  Update the labeled dataset via loss prediction-based uncertainty measurement

            # Randomly sample 10000 unlabeled data points
            random.shuffle(unlabeled_set)
            subset = unlabeled_set[:SUBSET]

            labeled_loader = DataLoader(cifar10_unlabeled, batch_size=BATCH,
                                          sampler=SubsetSequentialSampler(labeled_set),
                                          pin_memory=True)

            # Create unlabeled dataloader for the unlabeled subset
            unlabeled_loader = DataLoader(cifar10_unlabeled, batch_size=BATCH,
                                          sampler=SubsetSequentialSampler(subset),
                                          # more convenient if we maintain the order of subset
                                          pin_memory=True)

            # Measure uncertainty of each data points in the subset
            uncertainty = get_uncertainty(models, labeled_loader, unlabeled_loader)
            #print("main uncertainty: ", uncertainty)
            uncertainty = uncertainty.T
            # Index in ascending order
            arg = np.argsort(uncertainty).numpy().tolist()
            #print("main arg: ", arg) 
            
            # Update the labeled dataset and the unlabeled dataset, respectively
            labeled_set += list(torch.tensor(subset)[arg][-ADDENDUM:].numpy())  # select largest loss
            unlabeled_set = list(torch.tensor(subset)[arg][:-ADDENDUM].numpy()) + unlabeled_set[SUBSET:]

            # labeled_set += list(torch.tensor(subset)[arg][:ADDENDUM].numpy())  # select smallest influence
            # unlabeled_set = list(torch.tensor(subset)[arg][ADDENDUM:].numpy()) + unlabeled_set[SUBSET:]

            # Create a new dataloader for the updated labeled dataset
            dataloaders['train'] = DataLoader(cifar10_train, batch_size=BATCH,  # BATCH
                                              sampler=SubsetRandomSampler(labeled_set),
                                              pin_memory=True)

        # Save a checkpoint
        torch.save({
            'trial': trial + 1,
            'state_dict_backbone': models['backbone'].state_dict()
            # 'state_dict_module': models['module'].state_dict()
        },
            './cifar10/train/weights/active_resnet18_cifar10_trial{}.pth'.format(trial))

        print('---------------------------Current Trial is done-----------------------------',flush=True)


>> Train a Model...
Cycle: 1 Epoch: 1 --- Val Acc: 17.320 	 Best Acc: 17.320
Cycle: 1 Epoch: 21 --- Val Acc: 45.990 	 Best Acc: 45.990
Cycle: 1 Epoch: 41 --- Val Acc: 60.180 	 Best Acc: 60.180
Cycle: 1 Epoch: 61 --- Val Acc: 60.440 	 Best Acc: 60.440
Cycle: 1 Epoch: 81 --- Val Acc: 68.760 	 Best Acc: 68.760
Cycle: 1 Epoch: 101 --- Val Acc: 69.480 	 Best Acc: 69.480
Cycle: 1 Epoch: 121 --- Val Acc: 72.340 	 Best Acc: 72.340
Cycle: 1 Epoch: 141 --- Val Acc: 72.340 	 Best Acc: 72.340
Cycle: 1 Epoch: 161 --- Val Acc: 78.430 	 Best Acc: 78.430
Cycle: 1 Epoch: 181 --- Val Acc: 80.480 	 Best Acc: 80.480
Cycle: 1 Epoch: 200 --- Val Acc: 80.290 	 Best Acc: 80.480
>> Finished.
Trial 1/1 || Cycle 1/7 || Label set size 5000: Test acc 80.29
>> Start MMD:
>> Clustering Over:
>> Train a Model...
Cycle: 2 Epoch: 1 --- Val Acc: 77.140 	 Best Acc: 77.140
Cycle: 2 Epoch: 21 --- Val Acc: 77.910 	 Best Acc: 77.910
Cycle: 2 Epoch: 41 --- Val Acc: 76.780 	 Best Acc: 77.910
Cycle: 2 Epoch: 61 --- Val Acc: 73.